In [1]:
# !pip install mlflow
# !pip install unsloth
# !pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install numpy torch
# !pip uninstall bitsandbytes -y
# !pip install bitsandbytes
# !pip install accelerate
# !pip install datasets
# !pip install tf-keras

In [2]:
import pandas as pd
from datasets import load_dataset
#data
dataset = load_dataset(
    "csv",
    data_files="top_3000_longest_inputs.csv",
    split="train"
)
eval_dataset = load_dataset(
    "csv",
    data_files="cro_output_with_ner.csv",
)

/home/agata/Desktop/Faks_3_semestar/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(dataset)
print(eval_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 3000
})
DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 358
    })
})


In [4]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from unsloth import FastLanguageModel
# import torch
# import os

# # Disable wandb
# os.environ["WANDB_DISABLED"] = "true"

# # BitsAndBytes Configuration (4-bit Quantization)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.bfloat16,  # Use bfloat16
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
# )

# # Load Model with CPU Offloading
# model_name = "unsloth/Phi-3-mini-4k-instruct"
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=model_name,
#     max_seq_length=2048,
#     dtype=torch.bfloat16, 
#     load_in_4bit=True,
#     quantization_config=bnb_config,
#     device_map={"": "cuda:0", "lm_head": "cpu"},  # Offload output layer to CPU
#     llm_int8_enable_fp32_cpu_offload=True  # Enable offloading of large layers to CPU
# )

# print("Model loaded with CPU offloading!")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from transformers import BitsAndBytesConfig
import mlflow
import mlflow.pytorch
import os
# Model and tokenizer setup
from unsloth import FastLanguageModel
import torch

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"
# Initialize MLflow
mlflow.set_experiment("LLM Fine-Tuning")  # Replace with your desired experiment name

max_seq_length = 512 # 1024 # 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Phi-3-mini-4k-instruct",
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


2025-01-25 23:07:57.382365: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 23:07:57.391582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737842877.402848   54993 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737842877.406231   54993 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 23:07:57.418017: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


ImportError: cannot import name 'FastLanguageModelConfig' from 'unsloth' (/home/agata/Desktop/Faks_3_semestar/venv/lib/python3.12/site-packages/unsloth/__init__.py)

In [6]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "[The city and available attractions are {input}.]",
    conversation_extension = 1,
    output_column_name = "output",
)

from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [7]:
print(dataset)

Dataset({
    features: ['conversations'],
    num_rows: 3000
})


In [8]:
def format_conversations(examples):
    conversations = []
    for conversation in examples['conversations']:
        conversations.append(conversation)

    return {"conversations": conversations}

dataset = dataset.map(format_conversations, batched=True)


In [9]:
print(dataset)
print(dataset[0])

Dataset({
    features: ['conversations'],
    num_rows: 3000
})
{'conversations': [{'content': "[The city and available attractions are ('Nagykanizsa | other | Nagykanizsa Bus Station | Ajka, six per day, two hours, Alsópáhok, hourly, Aszófő, Badacsony, ten per day, one and half hours, Szigliget, six per day, one and half hours, Bak, Balatonakali, Balatonalmádi, Balatonboglár, three per day, one and half hours, Balatonederics, Balatonfüzfő, Balatongyörök, hourly, Balatonlelle, Balatonmáriafürdő, Balatonrendes, Balatonszentgyörgy, hourly, Balatonszepezd, Balatonudvari, Bánokszentgyörgy, hourly, Becehegy, hourly, hourly, Bikal, daily, Bocska, Bonyhád, Böhönye, hourly, Budapest, hourly, Celldömölk (daily, Cserszegtomaj, Csesztreg, Csörnyeföld, Csurgó, Devecser, Dióskál, Dobogómajor, Dunaföldvár, Egeraracsa, Egervár, Esztergályhorváti, Fakospuszta, Felsőbáránd, hourly, Felsőpáhok, hourly, Fenékpuszta, Fonyód, Galambok, hourly, Garabonc, Gellénháza, hourly, Gelse, Gyenesdiás, hourly, Győr,

In [14]:
chat_template = """Below describes some details about some city that can be visited.
Write a travel guide that mentions city characteristics.
Output text of travel guide.
>>> City Details:
{INPUT}
>>> Travel Guide:
{OUTPUT}"""
from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    default_system_message = "You are a helpful assistant that generates tourist guides based on input"
)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset=dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        # fp16=True,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 3000/3000 [00:00<00:00, 35743.05 examples/s]
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Map: 100%|██████████| 3000/3000 [00:00<00:00, 7408.09 examples/s]


In [15]:
from accelerate import Accelerator
import mlflow.pytorch
import torch

# Unwrap the model to ensure it's pickleable
accelerator = Accelerator()

# Unwrap the model
model = accelerator.unwrap_model(model)

# Optional: Manually disable AMP by setting the model to float32 (default precision)
model = model.to(torch.float32)  # Convert the model to float32

# Ensure no AMP optimization remains
for param in model.parameters():
    if param.grad is not None:
        param.grad.detach_()

model.zero_grad()

# Log the final model and tokenizer to MLflow
with mlflow.start_run():
    mlflow.log_params({
        "model_name": "unsloth/mistral-7b-v0.3-bnb-4bit",
        "learning_rate": trainer.args.learning_rate,
        "batch_size": trainer.args.per_device_train_batch_size,
        "num_train_epochs": trainer.args.num_train_epochs,
        "weight_decay": trainer.args.weight_decay,
    })

    # Log metrics (you can customize this to log additional metrics)
    eval_metrics = trainer.evaluate()
    mlflow.log_metrics(eval_metrics)

    print("Training and logging completed!")


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 3.71 GiB of which 19.56 MiB is free. Including non-PyTorch memory, this process has 3.62 GiB memory in use. Of the allocated memory 3.44 GiB is allocated by PyTorch, and 66.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": 'The passenger embarked from S.\n'\
                                'They are male.\n'\
                                'They have 1 siblings and spouses.\n'\
                                'Their passenger class is 3.\n'\
                                'Their age is 22.0.\n'\
                                'They paid $7.25 for the trip.'},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)